In [6]:
!pip -q install fastapi uvicorn nest_asyncio pyngrok==7.2.0 "pydantic<3" joblib scikit-learn


In [8]:
NGROK_AUTHTOKEN = "31dIffEzlumqxNYTNgdyPEwF6Xz_6DW1rQUikS4etab44vLrv"  # 있으면 넣고, 없으면 빈 문자열 유지


In [15]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 35.8 MB/s eta 0:00:00


In [28]:
import os, joblib, json
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import os, joblib, pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

# ===== KoNLPy tokenizer (Okt) =====
# 주의: joblib로 저장/재로딩할 때 이 함수 이름이 동일한 스코프에 존재해야 합니다.
# (새 세션에서 로드 시에도 아래 함수 정의가 있어야 로드가 성공합니다)
def tokenize_okt(text):
    try:
        from konlpy.tag import Okt
        if not hasattr(tokenize_okt, "_okt"):
            tokenize_okt._okt = Okt()
        # 정규화/어간추출 포함, 불필요 기호 제거는 TF-IDF의 token_pattern에서 처리
        return tokenize_okt._okt.morphs(str(text), norm=True, stem=True)
    except Exception:
        # KoNLPy 미설치/에러 시 공백 토큰화로 폴백 (성능 저하)
        return str(text).split()

# ===== 데이터 로딩 함수 =====
def load_data_csv(path="data_sent_new.csv", sample_n=2000, random_state=42):
    # 인코딩은 utf-8-sig 우선, 실패 시 cp949 시도
    encodings = ["utf-8-sig", "utf-8", "cp949"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except Exception as e:
            last_err = e
            df = None
    if df is None:
        raise last_err

    # 필수 컬럼 체크
    required_cols = {"sentence", "감정_대분류"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"CSV에 {missing} 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

    # 기쁨/불안만 사용
    df = df[df["감정_대분류"].isin(["기쁨", "불안"])].copy()
    if len(df) == 0:
        raise ValueError("기쁨/불안 라벨 행이 없습니다.")

    # 샘플링(2,000행, 행 수가 적으면 가능한 만큼만)
    n = min(sample_n, len(df))
    df = df.sample(n=n, random_state=random_state)

    X = df["sentence"].astype(str).tolist()
    y = df["감정_대분류"].astype(str).tolist()
    return X, y

MODEL_PATH = "/content/model.pkl"

if not os.path.exists(MODEL_PATH):
    # 1) 데이터 로드 & 전처리
    X, y = load_data_csv("data_sent_new.csv", sample_n=2000, random_state=42)

    # 2) 파이프라인: TF-IDF(OKT 토크나이저) + 디시전트리
    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(
            tokenizer=tokenize_okt,          # KoNLPy 기반 토크나이저
            token_pattern=r"(?u)\b\w+\b",    # 토크나이저가 있으므로 넓게 허용
            ngram_range=(1, 2),              # 유니그램+바이그램
            min_df=2,                        # 희귀 토큰 제거(필요시 조정)
            max_df=0.98
        )),
        ("clf", DecisionTreeClassifier(
            random_state=42,
            # 필요시 하이퍼파라미터 예: max_depth=30, min_samples_leaf=3 등
        ))
    ])

    # 3) 학습
    pipe.fit(X, y)

    # 4) 저장
    joblib.dump(pipe, MODEL_PATH)
    print(f"모델 학습 및 저장 완료: {MODEL_PATH}")
else:
    print(f"기존 모델을 로드합니다: {MODEL_PATH}")

# 5) 로딩(세션이 달라져서 토크나이저 오류가 난다면, 위의 tokenize_okt 정의가 반드시 있어야 함)
pipe = joblib.load(MODEL_PATH)

# 6) 확률 예측/클래스 확인
supports_proba = hasattr(pipe, "predict_proba")
label_classes = getattr(pipe, "classes_", None)
print("모델 로딩 OK. supports_proba:", supports_proba, "classes:", list(label_classes) if label_classes is not None else None)

# --- 사용 예시 ---
# samples = ["새 프로젝트 생각만 해도 설렌다.", "면접 전날이라 너무 불안해."]
# preds = pipe.predict(samples)
# probs = pipe.predict_proba(samples) if supports_proba else None
# print(preds, probs)


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


모델 학습 및 저장 완료: /content/model.pkl
모델 로딩 OK. supports_proba: True classes: [np.str_('기쁨'), np.str_('불안')]


In [29]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

app = FastAPI(title="TextClassifier API", version="1.0.0", description="PKL 모형 기반 문장 분류 API")
# CORS: 필요시 Custom GPT 도메인 허용(여기선 모두 허용)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

# ------- Request/Response 스키마 -------
class PredictRequest(BaseModel):
    text: str | None = None
    texts: list[str] | None = None  # 배치 요청 지원

class PredictResponse(BaseModel):
    labels: list[str]
    probabilities: list[dict] | None = None  # 각 문장별 {라벨: 확률}

# ------- 헬스체크 -------
@app.get("/health")
def health():
    return {"status": "ok"}

# ------- 예측 엔드포인트 -------
@app.post("/predict", response_model=PredictResponse)
def predict(req: PredictRequest):
    # 단문 또는 배치 둘 다 지원
    inputs = []
    if req.text is not None:
        inputs = [req.text]
    elif req.texts is not None and len(req.texts) > 0:
        inputs = req.texts
    else:
        return PredictResponse(labels=[], probabilities=[])

    preds = pipe.predict(inputs)
    labels = [str(p) for p in preds]

    probas = None
    if supports_proba:
        raw = pipe.predict_proba(inputs)  # shape: (n_samples, n_classes)
        classes = list(pipe.classes_) if label_classes is None else label_classes
        probas = []
        for row in raw:
            probas.append({cls: float(p) for cls, p in zip(classes, row)})

    return PredictResponse(labels=labels, probabilities=probas)

# --- Uvicorn을 백그라운드로 기동하는 헬퍼 ---
def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info", access_log=False)


In [30]:
import threading, time
from pyngrok import ngrok

# ngrok 준비
if NGROK_AUTHTOKEN:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 기존 터널 정리
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

# 새 HTTPS 터널
public_tunnel = ngrok.connect(8000, "http")
public_url = public_tunnel.public_url.replace("http://", "https://")  # https로 통일
print("Public URL:", public_url)

# servers 주입용 OpenAPI 오버라이드
from fastapi.openapi.utils import get_openapi

def custom_openapi():
    if app.openapi_schema:
        # 기존에 생성되어 있다면 servers만 바꿔치기
        app.openapi_schema["servers"] = [{"url": public_url}]
        return app.openapi_schema

    openapi_schema = get_openapi(
        title=app.title,
        version=app.version,
        description=app.description,
        routes=app.routes,
    )
    openapi_schema["servers"] = [{"url": public_url}]
    app.openapi_schema = openapi_schema
    return app.openapi_schema

app.openapi = custom_openapi  # 동적 바인딩

# Uvicorn 백그라운드 기동
thread = threading.Thread(target=run_uvicorn, daemon=True)
thread.start()
time.sleep(1.5)  # 서버 부팅 대기

print("Docs:", public_url + "/docs")
print("OpenAPI JSON:", public_url + "/openapi.json")
print("Health:", public_url + "/health")


INFO:     Started server process [378]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Public URL: https://30320357d6a0.ngrok.app
Docs: https://30320357d6a0.ngrok.app/docs
OpenAPI JSON: https://30320357d6a0.ngrok.app/openapi.json
Health: https://30320357d6a0.ngrok.app/health


In [32]:
import requests, json

sample = {"texts": ["오늘 정말 행복해!", "불안하고 걱정돼."]}
r = requests.post(public_url + "/predict", headers={"Content-Type":"application/json"}, data=json.dumps(sample))
print(r.status_code)
print(r.json())


200
{'labels': ['기쁨', '불안'], 'probabilities': [{'기쁨': 1.0, '불안': 0.0}, {'기쁨': 0.0, '불안': 1.0}]}
